In [175]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia numexpr rizaio

In [202]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.riza.command import ExecPython
from langchain.memory import ConversationBufferMemory

In [180]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPER_DEV_API_KEY")

llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize the DuckDuckGo search tool
ddg_search = DuckDuckGoSearchRun()

ddg_search_tool = Tool(
    name="DuckDuckGo Search",
    func=ddg_search.run,
    description="A tool for performing web searches using DuckDuckGo."
)

exec_python_tool = ExecPython()

# Load standard tools
standard_tools = load_tools(["llm-math","wikipedia","terminal"], llm=llm_openai,allow_dangerous_tools=True)

# Combine standard tools with the custom DuckDuckGo search tool
tools = standard_tools + [ddg_search_tool, exec_python_tool]

In [273]:
import re

class CustomExecPython(ExecPython):
    def run(self, tool_input: str, **kwargs):
        # Remove leading and trailing triple quotes (both single and double)
        tool_input = tool_input.strip("'''").strip('"""')

        # Decode unicode escape sequences (EXCEPT \n)
        tool_input = re.sub(r'\\([^n])', r'\1', tool_input)  # Decodes everything except '\n'
        
        print(f"Tool called: {tool_input}")  # Debugging Output
        return super().run(tool_input, **kwargs)

# Replace the existing exec_python_tool with the custom one
exec_python_tool = CustomExecPython()

# Update the tools list with the new exec_python_tool
tools = standard_tools + [ddg_search_tool, exec_python_tool]

In [274]:
test_cases = [
    "'''print('Hello, Riza!')'''",  # Triple-quoted string
    "\"\"\"print('Triple Quotes Test')\"\"\"",  # Another triple-quoted string variant
    "print('No Quotes Needed')",  # No triple quotes
    "x = 10\ny = 20\nprint(x + y)",  # Multi-line code
    "print(1/0)",  # Intentional error to test exception handling
]

for idx, test_code in enumerate(test_cases):
    try:
        print(f"\nTest Case {idx+1}:")
        print("Input Code:", test_code)
        output = exec_python_tool.run(test_code)
        print("Execution Output:", output)
    except Exception as e:
        print("Execution Error:", e)



Test Case 1:
Input Code: '''print('Hello, Riza!')'''
Tool called: print('Hello, Riza!')
Execution Output: Hello, Riza!


Test Case 2:
Input Code: """print('Triple Quotes Test')"""
Tool called: print('Triple Quotes Test')
Execution Output: Triple Quotes Test


Test Case 3:
Input Code: print('No Quotes Needed')
Tool called: print('No Quotes Needed')
Execution Output: No Quotes Needed


Test Case 4:
Input Code: x = 10
y = 20
print(x + y)
Tool called: x = 10
y = 20
print(x + y)
Execution Output: 30


Test Case 5:
Input Code: print(1/0)
Tool called: print(1/0)
Execution Output: Riza code execution returned a non-zero exit code. The output captured from stderr was:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ZeroDivisionError: division by zero



In [247]:
# What is a tool like this
for i in range(len(tools)):
    print(f"{tools[i].name}: {tools[i].description}")

Calculator: Useful for when you need to answer questions about math.
wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
terminal: Run shell commands on this MacOS machine.
DuckDuckGo Search: A tool for performing web searches using DuckDuckGo.
riza_exec_python: Execute Python code to solve problems.

    The Python runtime does not have filesystem access. You can use the httpx
    or requests library to make HTTP requests. Always print output to stdout.


In [286]:
# Define the prefix
PREFIX = """
Respond to the human as helpfully and accurately as possible.
Please do not repeat yourself. Start with the following format:
'''
Question: the input question you must answer
Thought: Do I need to use a tool?
'''
"""

# Define the format instructions
FORMAT_INSTRUCTIONS = """
Consider your actions before and after.

If your answer to 'Thought: Do I need to use a tool?' is 'No', continue with the following format:

'''
Thought: Do I need to use a tool? No.
Final Answer: [Your final answer here]
'''

If your answer to 'Thought: Do I need to use a tool?' is 'Yes', use the following format:

Thought: Do I need to use a tool? Yes.
Action: [The action to take, should be one of [{tool_names}]]
Action Input: [The input to the action]
Observation: [The result of the action]
... (this Thought/Action/Action Input/Observation can repeat N times until the answer becomes 'No')

After executing Python code, evaluate the **Observation**. If the output contains the computed result, print it as the **Final Answer**.
If an error occurs, attempt to correct it before re-executing.

This process should not loop indefinitely. The agent should **stop execution once the final result is obtained**.
"""

# Define the suffix
SUFFIX = '''
Begin!
Remember, you do not always need to use tools. Do not provide information the user did not ask for.

Question: {input}
Thought: {agent_scratchpad}
Chat History:{chat_history}
'''


In [287]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(tools, 
                         llm_openai, 
                         agent="zero-shot-react-description", 
                         handle_parsing_errors=True,
                         agent_kwargs={
                            'prefix': PREFIX,
                            'format_instructions': FORMAT_INSTRUCTIONS,
                            'suffix': SUFFIX,
                            'input_variables': ["input", "agent_scratchpad", "tool_names", "chat_history"],
                        },
                        memory = memory,
                        stop=["\nObservation:"],
                        max_iterations=10,  # Set to an appropriate number
                        verbose=True)

In [227]:
## Standard LLM Query
agent.run("Hi How are you today?")



> Entering new AgentExecutor chain...
'''
Question: Hi How are you today?
Thought: Do I need to use a tool? No.
Final Answer: I'm just a program, so I don't have feelings, but I'm here to help you! How can I assist you today?
'''

> Finished chain.


"I'm just a program, so I don't have feelings, but I'm here to help you! How can I assist you today?\n'''"

In [216]:
agent.run("Who is the United States President in 2025? What is his current age raised divided by 2?")



> Entering new AgentExecutor chain...
'''
Thought: Do I need to use a tool? Yes.
Action: DuckDuckGo Search
Action Input: "Who is the United States President in 2025?"
Observation: Trump defeated the Democratic presidential nominee, former Vice President Kamala Harris, in November's general election, after receiving 312 Electoral College votes to her 226. He also won the popular vote with 77.3 million votes, 49.9%, compared to Harris' 75 million, 48.4%. Harris attended the inaugural ceremony with her husband, Doug Emhoff. WASHINGTON − Donald Trump was sworn in Monday as the 47th president of the United States, returning to the White House after overcoming four criminal indictments and two assassination attempts ... Donald Trump was sworn in as the 47th President of the United States on Monday at the US Capitol. ... 2025 following President Donald Trump's pardons and commutations for defendants in the January ... Under provisions of the 20 th Amendment to the United States Constitution

"The United States President in 2025 is Donald Trump, and he was born on June 14, 1946. As of January 2025, he would be 78 years old. Therefore, his age divided by 2 is 39.\n'''"

In [168]:
agent.run("Create a blog article on Swami Vivekananda. Add 10 years to age of Swami vievekananda.")



> Entering new AgentExecutor chain...
'''
Question: Create a blog article on Swami Vivekananda. Add 10 years to age of Swami Vivekananda.
Thought: Do I need to use a tool? Yes.
Action: wikipedia
Action Input: "Swami Vivekananda"
Observation: Page: Swami Vivekananda
Summary: Swami Vivekananda (12 January 1863 – 4 July 1902), born Narendranath Datta was an Indian Hindu monk, philosopher, author, religious teacher, and the chief disciple of the Indian mystic Ramakrishna. He was a key figure in the introduction of Vedanta and Yoga to the Western world. He is credited with raising interfaith awareness and bringing Hinduism to the status of a major world religion in the late nineteenth century.
Born into an aristocratic Bengali Kayastha family in Calcutta, Vivekananda was inclined from a young age towards religion and spirituality. At the age of 18 he met Ramakrishna, later becoming a devoted follower and sannyasin (renunciate). After the death of Ramakrishna, Vivekananda toured the Indian

"Swami Vivekananda was born on January 12, 1863, and passed away on July 4, 1902, making him 39 years old at the time of his death. If we add 10 years to his age, he would be considered to have been 49 years old. \n\nSwami Vivekananda is an iconic figure in the introduction of Indian philosophies of Vedanta and Yoga to the Western world. He was a disciple of the mystic Ramakrishna and became a key proponent of Hinduism globally. His famous speech at the Parliament of Religions in Chicago in 1893 marked a significant moment in interfaith dialogue, promoting unity among different religions.\n\nVivekananda founded the Ramakrishna Mission and dedicated his life to social reform and the upliftment of society, emphasizing the importance of education and service. His teachings continue to inspire millions, and he is celebrated in India as a national hero and spiritual leader, with his birthday observed as National Youth Day. His legacy is characterized by a blend of spirituality, social respo

In [169]:
agent.run("Get me a summary of a wikipedia article on deepmind.")



> Entering new AgentExecutor chain...
'''
Thought: Do I need to use a tool? Yes.
Action: wikipedia
Action Input: "DeepMind"

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Google DeepMind
Summary: DeepMind Technologies Limited, trading as Google DeepMind or simply DeepMind, is a British-American artificial intelligence research laboratory which serves as a subsidiary of Alphabet Inc. Founded in the UK in 2010, it was acquired by Google in 2014 and merged with Google AI's Google Brain division to become Google DeepMind in April 2023. The company is headquartered in London, with research centres in the United States, Canada, France, and Germany.
DeepMind introduced neural Turing machines (neural networks that can access external memory like a conventional Turing machine), resulting in a computer that loosely resembles short-term memory in the human brain.
DeepMind has created neural network models to play video games and board games. It made headlines in 2016 after its AlphaGo program beat a human professional Go player Lee Sedol, a world champion, in a five-game match, which was the subject of a documentary film. A more general program

"DeepMind Technologies Limited, also known as Google DeepMind, is a British-American artificial intelligence research lab and a subsidiary of Alphabet Inc. Founded in 2010 and acquired by Google in 2014, it focuses on various AI advancements. Notably, it developed AlphaGo, which defeated a professional Go player, and AlphaFold, which made significant strides in protein folding. DeepMind continues to push the boundaries of AI research with applications across different domains.\n'''"

In [170]:
agent.run("What files are in my current directory?")



> Entering new AgentExecutor chain...
'''
Question: What files are in my current directory?
Thought: Do I need to use a tool? Yes.
Action: terminal
Action Input: lsExecuting command:
 ls

Observation: Week_1
Week_2
Week_3
Week_4
chroma_db
genai_env

Thought:

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Do I need to use a tool? No.
Final Answer: The files in your current directory are Week_1, Week_2, Week_3, Week_4, chroma_db, and genai_env.

> Finished chain.


'The files in your current directory are Week_1, Week_2, Week_3, Week_4, chroma_db, and genai_env.'

In [171]:
agent.run("Does my current directory or any of its sub-directories have a file about llama? If yes then list all of them.")



> Entering new AgentExecutor chain...
'''
Thought: Do I need to use a tool? Yes.
Action: terminal
Action Input: find . -type f -iname '*llama*'Executing command:
 find . -type f -iname '*llama*'


/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation: ./genai_env/lib/python3.10/site-packages/mlx_lm/models/__pycache__/llama.cpython-310.pyc
./genai_env/lib/python3.10/site-packages/mlx_lm/models/llama.py
./genai_env/lib/python3.10/site-packages/gptcache/adapter/llama_cpp.py
./genai_env/lib/python3.10/site-packages/gptcache/adapter/__pycache__/llama_cpp.cpython-310.pyc
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/llama_parity.py
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/__pycache__/llama_inputs.cpython-310.pyc
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/__pycache__/llama_torch.cpython-310.pyc
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/__pycache__/llama_parity.cpython-310.pyc
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/llama_torch.py
./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/llama_inputs.py
./genai_env/lib/python3

'Yes, your current directory and its sub-directories contain several files related to "llama". Here are the notable ones:\n\n1. `./genai_env/lib/python3.10/site-packages/mlx_lm/models/llama.py`\n2. `./genai_env/lib/python3.10/site-packages/gptcache/adapter/llama_cpp.py`\n3. `./genai_env/lib/python3.10/site-packages/onnxruntime/transformers/models/llama/llama_torch.py`\n4. `./Week_2/Day_4/Files/llama_paper.pdf`\n5. `./Week_4/Day_1/Fine_Tune_Llama_3_2_Vision_Model.ipynb`\n6. `./Week_3/Day_4/Llama_3_1_Fine_Tuning_Sri_Sri_RaviShankar.ipynb`\n7. `./Week_3/Day_5/Llama 3 Fine Tuning price predictions.ipynb`\n\nAnd many more files that include "llama" in their name across various directories.'

In [196]:
agent.run("What is the current price of nifty50?")



> Entering new AgentExecutor chain...
'''
Thought: Do I need to use a tool? Yes.
Action: DuckDuckGo Search
Action Input: "current price of Nifty 50"
Observation: NIFTY 50 Share Price Today, Live NSE Stock Price: Get the latest NIFTY 50 news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India. The National Stock Exchange of India (NSE) is the leading stock exchange of India offering live equity market watch updates including nifty, sensex today, share market live charts, stock market research reports and market trends today. Nifty 50 Today | Nifty 50 Live Updates- Nifty 50 Index, S&P CNX Nifty. Find today's trend for Nifty 50 Companies, News, Target Price, Stock Price, Stock Analysis Nifty 50 share price was down by -0.39% from the previous closing price of 23,696.30. W

'The current price of Nifty 50 is approximately 23,696.30, which is down by -0.39% from the previous closing price.'

In [293]:
agent.run("print the 100th fibinnachi number")



> Entering new AgentExecutor chain...
'''
Question: print the 100th fibinnachi number
Thought: Do I need to use a tool? Yes.
Action: riza_exec_python
Action Input: "def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nfibonacci(100)"Tool called: def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nfibonacci(100)

Observation: Riza code execution returned a non-zero exit code. The output captured from stderr was:
Traceback (most recent call last):
  File "<string>", line 1
    def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nfibonacci(100)
                      ^
SyntaxError: unexpected character after line continuation character

Thought:'''
Question: print the 100th fibinnachi number
Thought: Do I need to use a tool? Yes.
Action: riza_exec_python
Action Input: "def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b,

'Agent stopped due to iteration limit or time limit.'

In [292]:
agent.run("What is the date of birth of atal bihari vajpayee? Use web search for getting the year of birth. Calculate the number of seconds he lived using pythin and print the value.")



> Entering new AgentExecutor chain...
'''
Question: What is the date of birth of atal bihari vajpayee? Use web search for getting the year of birth. Calculate the number of seconds he lived using python and print the value.
Thought: Do I need to use a tool? Yes.
Action: DuckDuckGo Search
Action Input: "Atal Bihari Vajpayee date of birth"
Observation: Atal Bihari Vajpayee (born December 25, 1924, Gwalior, Madhya Pradesh, India—died August 16, 2018, New Delhi, Delhi) was the leader of the pro-Hindu Bharatiya Janata Party (BJP) and twice served as the prime minister of India (1996; 1998-2004).. Vajpayee was first elected to parliament in 1957 as a member of the Bharatiya Jana Sangh (BJS), a forerunner of the BJP. Atal Bihari Vajpayee's 100th Birth Anniversary: 25 December, 2024 marks the 100th birth anniversary of Atal Bihari Vajpayee, one of India's most respected leaders. This day, known as Good Governance Day, celebrates his leadership and the positive impact he had on the country. V

KeyboardInterrupt: 